# ARGO Data from 30.12.2017

We use the position and temperature from the argo float array:

 * http://www.argo.ucsd.edu/
 * Argo (2000). Argo float data and metadata from Global Data Assembly Centre (Argo GDAC). SEANOE. http://doi.org/10.17882/42182
  
The data was obtained from
 * ftp://ftp.ifremer.fr/ifremer/argo/geo



In [ ]:
import pooch
import xarray as xr

In [ ]:
logger = pooch.get_logger()
logger.setLevel("WARNING")

In [ ]:
def date_string(year: int, month: int, day: int) -> str:
    return f"{year}{month:02d}{day:02d}"


def get_data(year: int, month: int, day: int, ocean: str) -> str:

    oc = {"ATL": "atlantic", "IND": "indian", "PAC": "pacific"}

    date = date_string(year, month, day)

    path = (
        f"ftp://ftp.ifremer.fr/ifremer/argo/geo/{oc[ocean]}_ocean/{year}/{month:02d}/"
    )
    file = f"{date}_prof.nc"
    fname = f"ARGO_{ocean}_{date}.nc"

    return pooch.retrieve(
        path + file,
        known_hash=None,
        path="./rawdata/argo",
        fname=fname,
    )

In [ ]:
def process_argo_data(year: int, month: int, day: int, ocean: str):

    fN = get_data(year, month, day, ocean)
    ds = xr.open_dataset(fN)

    # only select lon, lat, temp
    ds = ds[["LONGITUDE", "LATITUDE", "TEMP"]]

    # only the topmost layer
    ds = ds.isel(N_LEVELS=0)

    ds = ds.rename(dict(LATITUDE="lat", LONGITUDE="lon"))

    date = date_string(year, month, day)
    fN_out = f"../data/ARGO_{ocean}_{date}.nc"

    ds.to_netcdf(fN_out, format="NETCDF4_CLASSIC")

    return ds


ATL = process_argo_data(2017, 12, 30, "ATL")
IND = process_argo_data(2017, 12, 30, "IND")
PAC = process_argo_data(2017, 12, 30, "PAC")